In [ ]:
# Example 8: calculating satellite positions in local coordinates
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget

import json
import matplotlib.pyplot as plt  
import gpslib_tutorial as gpslib
import numpy as np

SAT_LST = (24,19,12,15,13,17,22,25,23,10,32)
REC_POS = (4141657.6,604772.9,4796731.9)   
# lat,lon,height = (49.08293620495832, 8.30772631763587, 168.21197780780494)

timeStr = None
startPos,allGeo,allECEF = {},{},{}

with open('../data/230914_gpsFrames.json','r') as file: 
    FRAME_LST=json.load(file)  
    
for satNo in SAT_LST:
    satData = gpslib.SatData(satNo)
    satPos = gpslib.SatPos()
    sfLst = list(filter(lambda item:item['SAT']==satNo and 'ID' in item,
                        FRAME_LST))
    for sf in sfLst:
        satData.readSubframe(sf)
        if satData.ephemOk:
            break
    eph = satData.ephemData
    if timeStr is None:
        tow = satData.timeData[0][0]
        weekNum = eph['weekNum']
        timeStr = gpslib.gpsTimeStr(tow,weekNum) 
    x,y,z,_ = satPos.ecefCoord(tow,eph)
    theta,phi = gpslib.ecefToAzimElev(REC_POS,(x,y,z))  # (elevation,azimuth)
    startPos[satNo] = (theta,phi)
    
    step = 50
    t = tow-step
    geo,ecef = [],[]
    while theta > 0:
        t += step
        x,y,z,_ = satPos.ecefCoord(t,eph)
        ecef.append((t,x,y,z))
        theta,phi = gpslib.ecefToAzimElev(REC_POS,(x,y,z)) 
        geo.append((t,theta,phi))
    t = tow
    theta = 90
    while theta > 0:
        t -= step
        x,y,z,_ = satPos.ecefCoord(t,eph)
        ecef.append((t,x,y,z))
        theta,phi = gpslib.ecefToAzimElev(REC_POS,(x,y,z)) 
        geo.append((t,theta,phi))
    geo.sort()
    allGeo[satNo] = geo
    ecef.sort()
    allECEF[satNo] = ecef                

fig = plt.figure(figsize=(10,9))
fig.canvas.header_visible = False      
spec = fig.add_gridspec(3,2)
ax = fig.add_subplot(spec[0,0],projection='polar')
ax.set_title(timeStr,fontsize=10)
col = 0
for satNo in startPos:
    theta,phi = startPos[satNo]
    ax.plot(phi/180*np.pi,90-theta,'oC%d' % (col),ms=6,
            label='PRN %00d'%(satNo))    
    t,theta,phi = zip(*allGeo[satNo])
    theta = np.asarray(theta)
    phi = np.asarray(phi)
    plt.plot(phi/180*np.pi,90-theta,'-C%d' % (col),lw=1)  
    col = (col + 1) % 10   
ax.set_xticks(np.arange(0,2*np.pi,np.pi/4),
              ['N','NE','E','SE','S','SW','W','NW'])
ax.set_theta_zero_location("N")
ax.set_theta_direction('clockwise')
ax.set_ylim(0,90)
ax.set_yticks(range(0, 100, 10))                    
ax.set_yticklabels(['']+list(map(str, range(80, -10, -10))),fontsize=6) 
ax.legend(loc=(1.15,-0.1),fontsize='small')

bx = fig.add_subplot(spec[1,0])
cx = fig.add_subplot(spec[1,1])
dx = fig.add_subplot(spec[2,0])
ex = fig.add_subplot(spec[2,1])

vmin,vmax,amin,amax,col = 0,0,0,0,0
for satNo in allECEF:
    t,x,y,z = zip(*allECEF[satNo])
    t,x,y,z = np.asarray(t),np.asarray(x),np.asarray(y),np.asarray(z)
    r = np.sqrt((x-REC_POS[0])**2+(y-REC_POS[1])**2+(z-REC_POS[2])**2)
    
    v = np.diff(r)/(6*step)
    vmin,vmax = min(vmin,np.min(v)),max(vmax,np.max(v))
    bx.plot((t[:-1]-tow)/600,v,'-C%d' % (col),label='PRN %00d'%(satNo))

    a = np.diff(v)/(6*step)
    amin,amax = min(amin,np.min(a)),max(amax,np.max(a))
    cx.plot((t[:-2]-tow)/600,a,'-C%d' % (col),label='PRN %00d'%(satNo))

    f1 = 1575.42e6
    c = 3e8
    df = (np.sqrt((1+v/c)/(1-v/c))-1)*f1
    dx.plot((t[:-1]-tow)/600,df,'-C%d' % (col),label='PRN %00d'%(satNo))

    af = np.diff(df)/(6*step)
    ex.plot((t[:-2]-tow)/600,af,'-C%d' % (col),label='PRN %00d'%(satNo))
    
    col = (col + 1) % 10   
    
bx.set_xlabel('time $t-t_0$ in h')
bx.set_ylabel('radial velocity $v$ in m/s')
cx.set_xlabel('time $t-t_0$ in h')
cx.set_ylabel('radial acceleration $a$ in $m/s^2$')
dx.set_xlabel('time $t-t_0$ in h')
dx.set_ylabel('doppler frequency $\Delta f$ in Hz')
ex.set_xlabel('time $t-t_0$ in h')
ex.set_ylabel('frequency change $\dot{\Delta f}$ in Hz/s')

plt.tight_layout()                                 
plt.show()
    